# A python solution to predict the saturation pressure of a mixture using PR-EOS

***
**Note: Click on the `Cell` in the task bar above, then click on `Run All` to run the entire code**
---
**If you open this on myBinder, then click on `Run` above and then click on `Run all cells`**


In [1]:
import ipywidgets as widgets
import pandas as pd
import numpy as np
from IPython.display import display, HTML, Markdown
import math
import copy
import openpyxl as xl
from functools import reduce

import util
from library import render_table, render_multi_index_2D_table, drop_columns

## The cells below solves question 3

Calculate the saturation pressure for a crude oil sample from Assignment 2 for the following feeds 

|Feed|CH₄ (mol%)|C₃H₈ (mol%)|Heavy oil(mol%)|T °C|Pb KPa|Swelling factor|Viscosity at Pb (cp)|
|----|----|----|----|----|----|----|----|
|#1|0|57.94|42.06|20.7|664.9|1.304|31.6|
|#2|13.81|49.52|36.67|20.5|4368.5|1.362|0|

**This solution focuses on feed #1 which consist of only propane (C₃H₈) and a heavy oil sample**
---
You can select different feeds by choosing from the dropdown Feeds
1. Pseudos_1 is a mixture of propane and the heavy oil sample as one pseudocomponent
2. Pseudos_4 is a mixture of propane and the heavy oil sample as splitted and lumped into four (4) pseudocomponents
3. You can apply either the normal alpha function or the modified alpha function from the dropdown

#### Normal Alpha Function
![Normal Alpha Function](normal_alpha.PNG)

#### Modified Alpha Function (Li & Yang, 2011)
![Normal Alpha Function](modified_alpha.PNG)

In [2]:
# Working data
feeds = {
    "sample": {
        "sys_info": { "temp": util.to_rankine(100, 'f'), "Pbm": "", "swell": "", "visco": "" },
        "mixture": [
            { "component": "C\u2081", "name": "Methane", "xj": 0.7, "pc": 667.2, "tc": 343, "w": 0.008 },
            { "component": "n-C\u2085", "name": "n-Pentane", "xj": 0.3, "pc": 489.4, "tc": 845, "w": 0.251 }
        ],
        "bip_matrix": [
            [0, 0.021],
            [0.021, 0],
        ]
    },
    "Feed_1_PC_1": {
        "sys_info": { "temp": util.to_rankine(20.7, 'c'), "Pbm": util.to_psi(664.9, 'kpa'), "swell": 1.304, "visco": 31.6 },
        "mixture": [
            {"component": "C\u2083", "name": "Propane", "xj": 0.5794, "tc": util.to_rankine(206.06, 'f'), "pc": 616.0, "w": 0.1522, "Mw": 49.097,},
            {"component": "H_O", "name": "HeavyOil", "xj": 0.4206, "tc": util.to_rankine(903.2, 'k'), "pc": util.to_psi(15.917, 'atm'),"w": 0.8537, "Mw": 389,},
        ],
        "bip_matrix": [
            [0,0], # propane
            [0,0],  # HO
        ]
    },
    "Feed_2_PC_1": {
        "sys_info": { "temp": util.to_rankine(20.5, 'c'), "Pbm": util.to_psi(4368.5, 'kpa'), "swell": 1.362, "visco": 0 },
        "mixture": [
            {"component": "C\u2081", "name": "Methane", "xj": 0.1381, "tc": util.to_rankine(-116.67, 'f'), "pc": 666.4, "w": 0.0104, "Mw": 16.043,},
            {"component": "C\u2083", "name": "Propane", "xj": 0.4952, "tc": util.to_rankine(206.06, 'f'), "pc": 616.0, "w": 0.1522, "Mw": 49.097,},
            {"component": "H_O", "name": "HeavyOil", "xj": 0.3667, "tc": util.to_rankine(903.2, 'k'), "pc": util.to_psi(15.917, 'atm'),"w": 0.8537, "Mw": 389,},
        ],
        "bip_matrix": [
            [0, 0.00, 0], # methane
            [0.00, 0, 0], # propane
            [0, 0, 0],  # HO
        ]
    },
}

## Extracting the splits from Assignment 2 and lumping in 5 pseudocomponents 
The cell below extracts the splits from assignment 2 which is an excel sheet and then lumps the splits of the C51+ into five pseudocomponents and adds it as a feed to the above working data

In [13]:
wb = xl.load_workbook('Assignment_2.xlsx', data_only=True)
sheet = wb['For_assignment_two']
heavy_oil = {"xj": 0.4206, "Mw": 389, "SG": 0.9749}
heavy_oil['Kw'] = (3.33475*((heavy_oil['Mw'] ** 0.149495)/(heavy_oil['SG'] ** 0.935227))) + 3.099934

number_of_intended_pcs = 5

assignment_2_splits = []
for row in sheet.iter_rows(min_row=9, max_row=51, min_col=1, max_col=7, values_only=True):
    dict = {"component": row[0], "Mw": row[4], "zi": row[6],}
    assignment_2_splits.append(dict)
for row in sheet.iter_rows(min_row=58, max_row=124, min_col=1, max_col=7, values_only=True):
    dict = {"component": f'C{row[0]}', "Mw": row[3],"zi": row[1],}
    assignment_2_splits.append(dict)
    
def compute_SG_and_lumping_sum_limit(splits):
    lumping_sum_limiter = 0
    for comp in splits:
        comp['SG'] = (6.0108 * (comp['Mw'] ** 0.17947)) * (heavy_oil['Kw'] ** -1.18241)
        comp['zlnM'] = comp['zi'] * math.log(comp['Mw'])
        lumping_sum_limiter += comp['zlnM']
        
    return {"data": splits, "lumping_sum_limiter": lumping_sum_limiter }

computed_splits = compute_SG_and_lumping_sum_limit(assignment_2_splits)

def calculate_pcs(splits, number_of_pcs):
    data, lumping_sum_limiter = splits.values()
    limit_of_pc = lumping_sum_limiter / (number_of_pcs)
    
    current_sum = 0
    current_pc = 1
    for index, comp in enumerate(data):
        current_sum += comp['zlnM']
        comp['PC'] = current_pc
        if current_sum - limit_of_pc > 0:
            current_pc += 1
            current_sum = 0
    return data

# This is where you edit for the number of pseudocomponents of choice
computed_pcs = calculate_pcs(computed_splits, number_of_intended_pcs)

def group_pcs(pseudos_raw):
    groups = {}
    for comp in pseudos_raw:
        pc = comp['PC']
        if pc not in groups:
            groups[pc] = {"data": []}
        groups[pc]['data'].append(comp)
    
    # This for loop calculates the sum zL for each group
    for pc_key, value in groups.items():
        z_sum_pc = 0
        for comp in value['data']:
            z_sum_pc += comp['zi']
        value['z_sum'] = z_sum_pc
    
    # each group now has its z_sum
    final_return = []
    for pc_key, value in groups.items():
        Mw_sum = 0
        sum_denom_for_SG = 0
        for comp in value['data']:
            comp['z*'] = comp['zi'] / value['z_sum']
            Mw_sum += comp['z*'] * comp['Mw']
            sum_denom_for_SG += (comp['z*'] * comp['Mw']) / comp['SG']
            
        #  add the ML ans SG
        value['ML'] = Mw_sum
        value['SG'] = Mw_sum / sum_denom_for_SG
        
        # finally
        export_dict = {
            "component": f"PC_{pc_key}",
            "range": f"{value['data'][0]['component']} -  {value['data'][len(value['data'])-1]['component']}",
            "name": f"Pseudo_{pc_key}",
            "zi": value['z_sum'],
            "Mw": value['ML'],
            "SG": value['SG'],
        }
        final_return.append(export_dict)
    return final_return

pseudo_components = group_pcs(computed_pcs)

# now calculate critical properties for the PCs
def calculate_criticals(pseudos):
    pseudo_copy = pseudos[:]
    for comp in pseudo_copy:
        comp['Tb°R'] = (1928.3)-((1.695*(10**5))*(comp['Mw']**-0.03523)*(comp['SG']**3.266))*(math.exp(-((4.922*(10**-3))*(comp['Mw']))-(4.7685*comp['SG'])+((3.462*(10**-3))*(comp['Mw']*comp['SG']))))
        comp['tc'] = (341.7)+(811*comp['SG'])+((0.4244+(0.1174*comp['SG']))*(comp['Tb°R']))+((0.4669-(3.2623*comp['SG']))*(10**-5))*(comp['Tb°R']**-1)
        comp['pc'] = math.exp((8.3634-(0.0566*(comp['SG']**-1)))-(((0.24244+(2.2898*(comp['SG']**-1))+(0.11857*(comp['SG']**-2)))*(10**-3)) * (comp['Tb°R']) )+(((1.4685+(3.648*(comp['SG']**-1))+(0.47227*(comp['SG']**-2)))*(10**-7)))-(((0.42019+(1.6977*(comp['SG']**-2)))*(10**-10))*(comp['Tb°R']**3)))
        tbr_ = comp['Tb°R'] / comp['tc']
        if tbr_ < 0.8:
            comp['w'] = (-math.log(comp['pc']/14.7)+(-5.92714)+(6.09648*(tbr_**-1))+(1.28862*math.log(tbr_))+(-0.169347*math.log(tbr_**6)))/(15.2518+(15.6875*math.log(tbr_**-1))+(-13.4721*math.log(tbr_))+(0.43577*math.log(tbr_**6)))
        else:
            comp['w'] = (-7.904)+(0.1352*heavy_oil['Kw'])-(0.007465*(heavy_oil['Kw']**2))+(8.359*tbr_)+((1.408-(0.01063*heavy_oil['Kw']))**(tbr_**-1)) 
        
    return pseudo_copy

display(HTML(f"<h2>Lumping to {number_of_intended_pcs} pseudocomponents</h2>"))
display(pd.DataFrame(computed_pcs))
final_PC_properties = calculate_criticals(pseudo_components)
display(pd.DataFrame(final_PC_properties))

# Add the pseudos as a feed as feed_1_5_pcs
def create_a_new_feed_from_pcs(base_feed, pc_data):
    base_feed_copy = base_feed.copy()
    new_feed = {'sys_info': {}, 'mixture': [], 'bip_matrix': []}
    new_feed['sys_info'] = base_feed_copy['sys_info']
    heavy_oil_xj = 0
    
    for comp in base_feed_copy['mixture']:
        if comp['component'] != 'H_O':
            new_feed['mixture'].append(comp)
        elif comp['component'] == 'H_O':
            heavy_oil_xj = comp['xj']
            
    for comp in pc_data:
        temp_dict = { 'component': comp['component'], 'name': comp['name'], 'xj': comp['zi'] * heavy_oil_xj , 'tc': comp['tc'], 'pc': comp['pc'], 'w': comp['w'], 'Mw': comp['Mw'] }
        new_feed['mixture'].append(temp_dict)
        
    # bip matrix
    new_feed['bip_matrix'] = [[None for x in new_feed['mixture']] for y in new_feed['mixture']]
    for i in range(len(new_feed['mixture'])):
        for j in range(len(new_feed['mixture'])):
            new_feed['bip_matrix'][i][j] = 0
            
    return new_feed

created_feed = create_a_new_feed_from_pcs(feeds['Feed_1_PC_1'], final_PC_properties)
display(pd.DataFrame(created_feed['mixture']))
feeds[f"Feed_1_PC_{number_of_intended_pcs}"] = created_feed

,component,Mw,zi,SG,zlnM,PC,z*
0,C8,108,0.046304,0.781433,0.216802,1,0.177989
1,C9,122,0.031843,0.798716,0.152973,1,0.122401
2,C10,136,0.037450,0.814441,0.183981,1,0.143957
3,C11,150,0.046821,0.828889,0.234605,1,0.179978
4,C12,164,0.045317,0.842270,0.231109,1,0.174194
...,...,...,...,...,...,...,...
105,C113,1578,0.000093,1.264496,0.000683,5,0.000861
106,C114,1592,0.000087,1.266502,0.000644,5,0.000810
107,C115,1606,0.000082,1.268494,0.000606,5,0.000762
108,C116,1620,0.000077,1.270471,0.000571,5,0.000717


,component,range,name,zi,Mw,SG,Tb°R,tc,pc,w
0,PC_1,C8 - C13,Pseudo_1,0.260150,145.162007,0.825816,828.587262,1443.421366,241.511244,0.058120
1,PC_2,C14 - C18,Pseudo_2,0.224494,218.089800,0.887008,1006.913573,1593.252540,154.658016,0.045236
2,PC_3,C19 - C26,Pseudo_3,0.200232,304.139973,0.941769,1157.789364,1724.849836,108.909084,0.040411
3,PC_4,C27 - C43,Pseudo_4,0.187737,467.018186,1.017896,1341.165408,1896.674857,74.852022,0.039768
4,PC_5,C44 - C117,Pseudo_5,0.107780,818.206225,1.128455,1529.979413,2108.892485,58.401447,0.042455


,component,name,xj,tc,pc,w,Mw
0,C₃,Propane,0.579400,665.730000,616.000000,0.152200,49.097000
1,PC_1,Pseudo_1,0.109419,1443.421366,241.511244,0.058120,145.162007
2,PC_2,Pseudo_2,0.094422,1593.252540,154.658016,0.045236,218.089800
3,PC_3,Pseudo_3,0.084218,1724.849836,108.909084,0.040411,304.139973
4,PC_4,Pseudo_4,0.078962,1896.674857,74.852022,0.039768,467.018186
5,PC_5,Pseudo_5,0.045332,2108.892485,58.401447,0.042455,818.206225


## Widgets and subsequent PR-EOS

In [4]:
# widgets definitions
pressure_input = widgets.FloatText(description="psia (ass.)", value="100")
feed_options = [(value, index) for index, value in enumerate(feeds.keys())]
feed_dd = widgets.Dropdown(options=feed_options, value=1, description="Feeds")
alpha_dd = widgets.Dropdown(options=[('normal', 0), ('Modified', 1)], value=0, description="Alpha fxn")

current_feed = {}
current_feed = feeds[list(feeds.keys())[feed_dd.value]]

In [5]:
def step_1_k_calc(mixture, pressure, temp):
    mixture_copy = mixture[:]
    #     usng wilsons correlation
    def calc_k_for_each(pci, tci, wi):
        term_1 = pci / pressure
        term_2 = (1 + wi)
        term_3 = (1 - (tci / temp))
        return term_1 * math.exp(5.37 * term_2 * term_3)
    
    for comp in mixture_copy:
        comp['k'] = calc_k_for_each(comp['pc'], comp['tc'], comp['w'])
        comp['yj'] = comp['k'] * comp['xj']
        
    return mixture_copy

In [6]:
# Calculate each component PR-EOS Properties
def step_2_eos_props(mixture, temp, pressure, alpha_type):
    mixture_copy = mixture[:]
    
    for comp in mixture_copy:
        pc = comp['pc']
        tc = comp['tc']
        w = comp['w']
        tr = round(util.calculate_tr(temp, tc), 5)
        comp['tr'] = tr
        b = round(util.calculate_b(tc, pc), 4)
        comp['b'] = b
        ac = round(util.calculate_ac(tc, pc), 0)
        comp['ac'] = ac
        alpha = round(util.calculate_alpha(w, temp, tc, alpha_type), 4)
        comp['alpha'] = alpha
        aT = round(util.calculate_at(ac, alpha), 0)
        comp['aT'] = aT
        
    return mixture_copy

In [7]:
def step_3_phase_calculation(mixture, feed_dd_value, pressure, temp):
    mixture_copy = mixture[:]
    phase_data = { "liquid": {}, "gas": {} }
    
    aTjs = util.keys_to_list_of_dict('aT', mixture_copy)
    aTjs_list = list(aTjs.values())
    bjs = util.keys_to_list_of_dict('b', mixture_copy)
    bjs_list = list(bjs.values())
    current_bip = feeds[list(feeds.keys())[feed_dd_value]]['bip_matrix']
    
    # for liquid phase
    xjs = util.keys_to_list_of_dict('xj', mixture_copy)
    xjs_list = list(xjs.values())
    aT_liquid = util.calculate_aT_per_phase(xjs_list, aTjs_list, current_bip)
    b_liquid = util.calcuate_b_per_phase(xjs_list, bjs_list)
    A_liquid = util.calculate_A(aT_liquid, pressure, temp)
    B_liquid = util.calculate_B(b_liquid, pressure, temp )
    liquid = { 'aT': aT_liquid, 'b': b_liquid, "A": A_liquid, "B": B_liquid }
    phase_data['liquid'] = liquid
    
    # for gas phase
    yjs = util.keys_to_list_of_dict('yj', mixture_copy)
    yjs_list = list(yjs.values())
    aT_gas = util.calculate_aT_per_phase(yjs_list, aTjs_list, current_bip)
    b_gas = util.calcuate_b_per_phase(yjs_list, bjs_list)
    A_gas = util.calculate_A(aT_gas, pressure, temp)
    B_gas = util.calculate_B(b_gas, pressure, temp )
    gas = { 'aT': aT_gas, 'b': b_gas, "A": A_gas, "B": B_gas }
    phase_data['gas'] = gas
    
    return phase_data

In [8]:
def step_4_z_factor(phase_data):
    phase_data_copy = phase_data.copy()
    
    liquid_phase_roots = util.cubic_root_calculation_for_PR_EOS(phase_data_copy['liquid']['A'], phase_data_copy['liquid']['B'], True)
    liquid_real_roots = util.filter_out_complex_numbers(liquid_phase_roots)
    z_liquid = util.get_highest_or_lowest_root(liquid_real_roots, 'liquid')
    phase_data_copy['liquid']['z'] = z_liquid
    
    gas_phase_roots = util.cubic_root_calculation_for_PR_EOS(phase_data_copy['gas']['A'], phase_data_copy['gas']['B'], True)
    gas_real_roots = util.filter_out_complex_numbers(gas_phase_roots)
    z_gas = util.get_highest_or_lowest_root(gas_real_roots, 'gas')
    phase_data_copy['gas']['z'] = z_gas
    
    return phase_data_copy

In [9]:
def step_5_composition_coef(comp_list, phase_data, feed_dd_value):
    current_bip = feeds[list(feeds.keys())[feed_dd_value]]['bip_matrix']
    comp_list_copy = copy.deepcopy(comp_list)
    
    def calculate_summation(target_comp, phase, bip_to_use):
        """The phase can either be xj for liquid and yj for gas"""
        sum = 0
        for ind, value in enumerate(comp_list_copy):
            sum += value[phase] * ((target_comp['ac'] * value['ac'] * target_comp['alpha'] * value['alpha']) ** 0.5) * (1 - bip_to_use[ind])
        return sum
    
    for index, comp in enumerate(comp_list_copy):
        liquid_Bj = comp['b'] / phase_data['liquid']['b']
        liquid_Aj = (2 * calculate_summation(comp, 'xj', current_bip[index])) / phase_data['liquid']['aT']
        comp['liquid'] = {'Aj': liquid_Aj, 'Bj': liquid_Bj}
        
        gas_Bj = comp['b'] / phase_data['gas']['b']
        gas_Aj = (2 * calculate_summation(comp, 'yj', current_bip[index])) / phase_data['gas']['aT']
        comp['gas'] = {'Aj': gas_Aj, 'Bj': gas_Bj}
    
    new_list = [{'component': d['component'], 'liquid': d['liquid'], 'gas': d['gas']} for d in comp_list_copy]
    return new_list

In [10]:
def step_6_fugacity_coef(component_coef_data, original_data, phase_data):
    merged = [dict1 | dict2 for dict1 in component_coef_data for dict2 in original_data if dict1['component'] == dict2['component']]
    
    def solve_fug_coef(comp, phase):
        term_1 = comp[phase]['Bj'] * (phase_data[phase]['z'] - 1)
        term_2 = math.log(phase_data[phase]['z'] - phase_data[phase]['B'])
        term_3 = (phase_data[phase]['A'] / ((2**1.5) * phase_data[phase]['B']))
        term_4 = (comp[phase]['Aj'] - comp[phase]['Bj'])
        numerator = phase_data[phase]['z'] + (((2 ** 0.5) + 1) * phase_data[phase]['B'])
        denom = phase_data[phase]['z'] - (((2 ** 0.5) - 1) * phase_data[phase]['B'])
        term_5 = math.log(numerator / denom)
        
        result = math.exp(term_1 - term_2 - ( term_3 * term_4 * term_5 ))
        return result
    fug_coef_data = []
    for component in merged:
        fug_coef_l = solve_fug_coef(component, 'liquid')
        fug_coef_g = solve_fug_coef(component, 'gas')
        
        fug_coef_data.append({"component": component['component'], "fug_coef_l": fug_coef_l, "fug_coef_g": fug_coef_g})
    return fug_coef_data

In [11]:
def step_7_fugacity_k_and_error(fug_coef_data, original_data, pressure):
    merged = [dict1 | dict2 for dict1 in fug_coef_data for dict2 in original_data if dict1['component'] == dict2['component']]
    
    next_iteration_data = []
    previous_iteration_data = []
    err_sum = 0
    new_pressure = 0
    for comp in merged:
        fug_l = comp['fug_coef_l'] * (comp['xj'] * pressure)
        fug_g = comp['fug_coef_g'] * (comp['yj'] * pressure)
        err = (1 - (fug_l / fug_g)) ** 2
        err_sum += err
        
        k_new = comp['fug_coef_l'] / comp['fug_coef_g']
        yj_new = comp['xj'] * k_new
        new_pressure += fug_l / comp['fug_coef_g']
        
        previous_iteration_data.append({
            "component": comp['component'],
            "xj": comp['xj'],
            "yj": comp['yj'],
            "k": comp['k'],
            "fug_coef_l": comp['fug_coef_l'], 
            "fug_coef_g": comp['fug_coef_g'],
            "fug_l": fug_l,
            "fug_g": fug_g,
            "ε": err,
        })
        next_iteration_data.append({
            "component": comp['component'],
            "xj": comp['xj'],
            "k_new": k_new,
            "yj_new": yj_new,
        })
        
    return {
        "next": next_iteration_data,
        "previuos": previous_iteration_data,
        "error": err_sum, 
        "next_pressure": round(new_pressure, 1),
        "previous_pressure": pressure
    }
        

In [12]:
def on_any_change(feed_dd_value, pressure, alpha_dd_value):
    """Alpha fucntion: 0 = normal, 1 = modified"""
    
    current_feed = feeds[list(feeds.keys())[feed_dd_value]]
    mixture = copy.deepcopy(current_feed['mixture'])
    temp = current_feed['sys_info']['temp']
    display_tables = [0,1,2,3,4,5,6,7,8,9]
    
    display_tables[0] = pd.DataFrame.from_dict([current_feed['sys_info']])
    display_tables[1] = pd.DataFrame(mixture)
    
    # The first time to get k_values via wilson
    k_mixture = step_1_k_calc(mixture, pressure, temp)
    display_tables[2] = pd.DataFrame(k_mixture)
    
    loop_pressure = pressure
    error = 1000000000000000
    number_of_loops = 0
    while error > (10**-10):
        number_of_loops += 1
        loop_initial_k_mixture = k_mixture[:]
        
        eos_props_mixture = step_2_eos_props(loop_initial_k_mixture, temp, loop_pressure, alpha_dd_value)
        display_tables[3] = pd.DataFrame(eos_props_mixture)

        phase_data = step_3_phase_calculation(eos_props_mixture, feed_dd_value, loop_pressure, temp)
        display_tables[4] = pd.DataFrame(phase_data).transpose()

        phase_data_with_z = step_4_z_factor(phase_data)
        display_tables[5] = pd.DataFrame(phase_data_with_z).transpose()

        component_coef_data = step_5_composition_coef(eos_props_mixture, phase_data, feed_dd_value)
        component_coef_data_df = render_multi_index_2D_table(component_coef_data, 'liquid', 'gas', 'Aj', 'Bj', 'component', False)
        display_tables[6] = component_coef_data_df

        fugacity_coef_data = step_6_fugacity_coef(component_coef_data, eos_props_mixture, phase_data_with_z)
        display_tables[7] = pd.DataFrame(fugacity_coef_data)

        fugacity_data = step_7_fugacity_k_and_error(fugacity_coef_data, eos_props_mixture, loop_pressure)
        display_tables[8] = pd.DataFrame(fugacity_data['previuos'])
        display_tables[9] = pd.DataFrame({"Final Pressure": [fugacity_data['previous_pressure']]}).transpose()
        
        # update for the loop
        error = fugacity_data['error']
        loop_pressure = fugacity_data['next_pressure']
        
        # K_mixture is the mixture from after wilson
        for comp in loop_initial_k_mixture:
            for new_comp in fugacity_data['next']:
                if comp['component'] == new_comp['component']:
                    comp['k'] = new_comp['k_new']
                    comp['yj'] = new_comp['yj_new']
                    
    for index, each_df in enumerate(display_tables):
        if index == len(display_tables) - 2:
            display(HTML("<h2>Final Results from most recent iteration</h2>"))
        display(each_df)
    print(f"Loops: {number_of_loops}")
    
interactive_widgets = widgets.interactive(
    on_any_change,
    feed_dd_value=feed_dd,
    pressure=pressure_input,
    alpha_dd_value=alpha_dd
)
display(HTML("<h2>Predicting the saturation pressure of a mixture</h2>"))
display(interactive_widgets)

interactive(children=(Dropdown(description='Feeds', index=1, options=(('sample', 0), ('Feed_1_PC_1', 1), ('Fee…

#### References

Li, H., & Yang, D. (2011). Modified α function for the Peng−Robinson equation of state to improve the vapor pressure prediction of non-hydrocarbon and hydrocarbon compounds. Energy Fuels, 25(1), 215-223. doi:10.1021/ef100927z